### Direct Marketing in Banking - Propensity Modelling with Tabular Data

# Part 1: SageMaker Autopilot and AutoGluon

> *This notebook works well with the `Python 3 (Data Science 3.0)` kernel on SageMaker Studio*

This workshop explores a tabular, [binary classification](https://en.wikipedia.org/wiki/Binary_classification) use-case with significant **class imbalance**: predicting which of a bank's customers are likely to respond to a targeted marketing campaign.

In this first notebook, you'll tackle the challenge with two AutoML tools: [Amazon SageMaker Autopilot](https://aws.amazon.com/sagemaker/autopilot/) and SageMaker's [built-in AutoGluon-Tabular algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/autogluon-tabular.html).

## Contents

> ℹ️ **Tip:** You can use the Table of Contents panel in the left sidebar on JupyterLab / SageMaker Studio, to view and navigate sections

1. **[Prepare our environment](#Prepare-our-environment)**
1. **[Fetch the example dataset](#Fetch-the-example-dataset)**
1. **[Starting fast with SageMaker Autopilot](#Starting-fast-with-SageMaker-Autopilot)**
1. **[Diving deeper with AutoGluon-Tabular](#Diving-deeper-with-AutoGluon-Tabular)**
1. **[Conclusions](#Conclusions)**

## Prepare our environment

To get started, we'll need to:

- **Import** some useful libraries (as in any Python notebook)
- **Configure** -
    - The [Amazon S3 bucket](https://docs.aws.amazon.com/AmazonS3/latest/userguide/Welcome.html#CoreConcepts) and folder where **data** should be stored (to keep our environment tidy)
    - The [IAM role](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles.html) defining what **permissions** the jobs you create will have
- **Connect** to AWS in general (with [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html)) and SageMaker in particular (with the [sagemaker SDK](https://sagemaker.readthedocs.io/en/stable/)), to use the cloud services

Run the cell below, to set these up.

> ℹ️ **Tip:** Just like in a regular [JupyterLab notebook](https://jupyterlab.readthedocs.io/en/stable/user/interface.html), you can run code cells by clicking in to target cell - and then pressing the play (▶️) button in the toolbar or `Shift+Enter` on the keyboard.

In [ ]:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
import json
import time

# External Dependencies:
import boto3  # General-purpose AWS SDK for Python
import numpy as np  # For matrix operations and numerical processing
import pandas as pd  # Tabular data utilities
import sagemaker  # High-level SDK specifically for Amazon SageMaker
from sagemaker.automl.automl import AutoML as AutoMLEstimator
from sagemaker.feature_store.feature_group import FeatureGroup

# Local Helper Functions:
import util

# Setting up SageMaker parameters
sgmk_session = sagemaker.Session()  # Connect to SageMaker APIs
bucket_name = sgmk_session.default_bucket()  # Select an Amazon S3 bucket
bucket_prefix = "sm101/direct-marketing"  # Location in the bucket to store our files
sgmk_role = sagemaker.get_execution_role()  # IAM Execution Role to use for permissions

print(f"s3://{bucket_name}/{bucket_prefix}")
print(sgmk_role)

## Fetch the example dataset

This example uses the [UCI Bank Marketing Dataset](https://archive.ics.uci.edu/ml/datasets/bank+marketing) as per: S. Moro, P. Cortez and P. Rita. *A Data-Driven Approach to Predict the Success of Bank Telemarketing.* Decision Support Systems, Elsevier, 62:22-31, June 2014.

In the following cells we'll download the dataset locally, store it in Amazon S3, and **also** load a transformed copy into [Amazon SageMaker Feature Store](https://aws.amazon.com/sagemaker/feature-store/).

> ℹ️ **Tip:** You can train and deploy models in SageMaker **without using** SageMaker Feature Store, but we introduce it in this example to show you to a wider range of SageMaker features.
>
> Don't worry too much about the details of how the data loading is done here - but for the curious, you can check out the code behind these helper functions in  [util/data.py](util/data.py).

In [ ]:
raw_data_path = util.data.fetch_sample_data()
print(f"Got: {raw_data_path}\n")

print("Uploading raw dataset to Amazon S3:")
raw_data_s3uri = f"s3://{bucket_name}/{bucket_prefix}/raw.csv"
!aws s3 cp {raw_data_path} {raw_data_s3uri}

In [ ]:
%%time
feature_group_name = "sm101-direct-marketing"
print("Loading data to SageMaker Feature Store:")

# No need to re-run this if you've done it already - just set `feature_group_name` variable.
util.data.load_sample_data(
    raw_data_path,
    fg_s3_uri=f"s3://{bucket_name}/{bucket_prefix}/feature-store",
    feature_group_name=feature_group_name,
    ignore_cols=[
        "duration", "emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"
    ],
)

> ⏰ **You don't have to wait** for this cell to finish running: As soon as you reach the `Ingesting data...` step, you're ready to continue on to the next section!

▶️ As soon as you reach the `Ingesting data...` stage, you'll be able to see your "feature group" in the SageMaker Feature Store catalog:

- Select the triangular *SageMaker Resources* icon from the left sidebar in SageMaker Studio
- Choose `Feature Store` from the drop-down menu and click `Open Feature store`

Note you can explore the catalog either by "feature group" (table), or searching for individual features themselves. Descriptions and some tags have already been populated for you, based on the dataset description from UCI.

![](img/feature-store-features.png "Screenshot of SMStudio Feature Store UI showing feature catalog")

## Starting fast with SageMaker Autopilot

> ℹ️ **Tip: To skip the following manual steps** - Scroll down and you'll find code to create a similar setup through the API.

Autopilot makes it easy to get started on tabular ML problems, even without extensive data preparation or writing any code. This is because:

- Autopilot will automatically explore multiple data pre-processing options, algorithms, and hyperparameters for you - to identify a high-performing model
- Even if you **do** want to perform some manual feature engineering first, Autopilot has direct integrations from [SageMaker Data Wrangler](https://aws.amazon.com/sagemaker/data-wrangler/) (SageMaker's low-code/no-code data preparation tool). You could explore this by creating a new Data Wrangler Flow from the launcher or File > New > Data Wrangler Flow.

▶️ While your data finishes importing to SageMaker Feature Store, let's **start an Autopilot experiment using the raw CSV file**

1. **Select** the triangular ▽ *SageMaker Resources* icon from the left sidebar in SageMaker Studio
1. **Choose** `Experiments and trials` from the drop-down menu
1. **Click** the `Create Autopilot Experiment` button
    - If you don't see this button, you've probably drilled down into an experiment/trial: Look for the 🏡 home icon button just below the "Experiments and trials" drop-down, and click it to return to the top level.

▶️ In the first **Experiment and data details** step:

- Choose an **Experiment name** - something like `sm101-autopilot-1` should be fine.
- For **S3 location**, you can use the `Browse` button or just enter the URI from `raw_data_s3uri` earlier in this notebook.
- Leave any other settings as default and click **Next**

![](img/autopilot-01-select-data.png "Screenshot of Autopilot workflow selecting the raw data CSV in S3")

▶️ In the next **Experiment and data details** step:

- For the **Target** column, select `y`
- **De-select** the features from `ignore_cols` earlier in this notebook, to **exclude** them from the model.
    - ⚠️ As discussed on the [UCI dataset page](https://archive.ics.uci.edu/ml/datasets/bank+marketing), the `duration` field is particularly inappropriate to include in modelling as it leaks information about the target variable (calls with longer duration were more likely to be successful sales, and call duration cannot be known at the point of targeting which customers to approach. We also exclude the final 5 macro-economic variables as these may not be easily available at inference time.

![](img/autopilot-02-select-features.png "Screenshot of Autopilot workflow excluding ignored features and selecting y as target variable")

▶️ In the next **Training method** step:

- You can leave the default **Auto** selection method. Since this dataset is small, the Ensembling method will be automatically selected. If you like, you could create a second Experiment to compare both methods.

![](img/autopilot-03-select-mode.png "Select Auto mode in the UI, which will be equivalent to Ensembling for this small dataset")

▶️ In the next **Deployment and advanced settings** step:

- **IF you selected "HPO" training method in the previous screen**: Consider reducing the `Runtime > Max candidates` setting from 250 to ~50. The default setting would likely take several hours to train.
- **Otherwise**, leave all settings as default

▶️ **Click next**, review your settings, and then **Create experiment**.

![](img/autopilot-04-runtime.png "Screenshot selecting smaller number of max candidates in runtime menu - only applicable for HPO jobs")

Alternatively, you may like to create Autopilot jobs through the [CreateAutoMLJob API](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_CreateAutoMLJob.html) or - as shown below - the high-level `AutoML` class in the [SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/api/training/automl.html):

> ⚠️ You don't need to run the following cell if you already created an Autopilot job manually!

In [ ]:
# There's no need to run this cell if you created an Autopilot job manually!
autopilot = AutoMLEstimator(
    role=sgmk_role,
    target_attribute_name="y",

    # At the time of writing, the high-level Python SDK didn't support ensembling mode - so we'll
    # use the HPO mode instead with limited max_candidates:
    max_candidates=20,

    # Optional params to keep the environment tidy:
    base_job_name="sm101-autopilot",
    output_path=f"s3://{bucket_name}/{bucket_prefix}/autopilot",
)

autopilot.fit(raw_data_s3uri, wait=False)

## Diving deeper with AutoGluon-Tabular

Another useful tool to build highly-accurate models quickly is the open-source [AutoGluon framework](https://auto.gluon.ai/stable/index.html) and the SageMaker built-in [AutoGluon-Tabular algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/autogluon-tabular.html).

As outlined in the [2020 paper by Erickson, Mueller et al](https://arxiv.org/abs/2003.06505), AutoGluon-Tabular is an advanced model stacking ensembling framework that beat 99% of participating data scientists in benchmark Kaggle contests with just 4hrs of model training.

In fact at the time of writing, SageMaker Autopilot makes use of AutoGluon under the hood when running in ensembling mode: But you can also use AutoGluon directly as shown here for more customized experiments.


### Understand the algorithm requirements

The first step to using any SageMaker built-in algorithm is understanding its overall characteristics and the interface it offers. Here we'll refer to:

- The [algorithm docs](https://docs.aws.amazon.com/sagemaker/latest/dg/autogluon-tabular.html) to understand the **detail** of the **data formats** and **(hyper)-parameters** it supports - as well as sample notebooks
- The [Common Parameters doc](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html) to compare the **high-level configurations** and capabilities between algorithms.

For some built-in algorithms, a container `image_uri` is enough to configure. However as described in the [how to use](https://docs.aws.amazon.com/sagemaker/latest/dg/autogluon-tabular.html#autogluon-tabular-modes) page, [SageMaker JumpStart-based](https://sagemaker.readthedocs.io/en/stable/overview.html#use-built-in-algorithms-with-pre-trained-models-in-sagemaker-python-sdk) algorithms like AutoGluon-Tabular also need a `script_uri` and `model_uri`.

These resources are all pre-built, and we can look them up by the `retrieve()` functions in the SageMaker Python SDK as shown below.

The default [hyperparameters](https://docs.aws.amazon.com/sagemaker/latest/dg/autogluon-tabular-hyperparameters.html) for the algorithm can also be loaded through the SDK, and below we make some minor customizations ready for inference.

In [ ]:
from sagemaker import image_uris, script_uris, model_uris
from sagemaker.hyperparameters import retrieve_default as retrieve_default_hyperparams

ag_model_id, ag_model_version, train_scope = (
    "autogluon-classification-ensemble",
    "*",
    "training",
)
training_instance_type = "ml.p3.2xlarge"

# Retrieve the docker image
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    model_id=ag_model_id,
    model_version=ag_model_version,
    image_scope=train_scope,
    instance_type=training_instance_type,
)
print(train_image_uri)
# Retrieve the training script
train_source_uri = script_uris.retrieve(
    model_id=ag_model_id, model_version=ag_model_version, script_scope=train_scope
)
print(train_source_uri)
# Retrieve the pre-trained model tarball to further fine-tune. In tabular case, however, the pre-trained model tarball is dummy and fine-tune means training from scratch.
train_model_uri = model_uris.retrieve(
    model_id=ag_model_id, model_version=ag_model_version, model_scope=train_scope
)
print(train_model_uri)

# Retrieve the default hyper-parameters for training the model
hyperparameters = retrieve_default_hyperparams(
    model_id=ag_model_id, model_version=ag_model_version
)

# [Optional] Override default hyperparameters with custom values
hyperparameters["auto_stack"] = "True"
hyperparameters["save_space"] = "True"
print("\n", hyperparameters)

### Extract batch data from the SageMaker Feature Store

Next, we'll extract a snapshot of data from the (offline/batch) SageMaker Feature Store via serverless SQL query with [Amazon Athena](https://aws.amazon.com/athena/), to prepare for model training.

Feature Store **tracks the history** of records, allowing you to reproduce point-in-time snapshots even when features change over time.

- **Example queries** for time-travel and other views are available through the SageMaker Studio Feature Store UI: From your Feature Group, switch to the "Sample queries" tab.
- The additional `event_time`, `write_time`, `api_invocation_time`, `is_deleted` and `row_number` fields returned in the below query are metadata for this history tracking - so won't be used in the actual model training.

In [ ]:
feature_group = FeatureGroup(feature_group_name, sagemaker_session=sgmk_session)
query = feature_group.athena_query()
table_name = query.table_name

data_extract_s3uri = f"s3://{bucket_name}/{bucket_prefix}/data-extract"
!aws s3 rm --quiet --recursive {data_extract_s3uri}  # Clear any previous extractions
print(f"Querying feature store to extract snapshot at:\n{data_extract_s3uri}")
query.run(
    f"""
    SELECT *
    FROM
        (SELECT *,
        row_number()
        OVER
            (PARTITION BY "customer_id"
            ORDER BY "event_time" DESC, Api_Invocation_Time DESC, write_time DESC)
        AS row_number
        FROM "sagemaker_featurestore"."{table_name}"
        WHERE "event_time" <= {time.time()})
    WHERE row_number = 1 AND NOT is_deleted;
    """,
    output_location=data_extract_s3uri,
)
query.wait()

full_df = query.as_dataframe()
print(f"Got {len(full_df)} records")
full_df

### Split and prepare datasets

From the [Input and Output Interface section](https://docs.aws.amazon.com/sagemaker/latest/dg/autogluon-tabular.html#InputOutput-AutoGluon-Tabular) of the algorithm doc, we know that AutoGluon-Tabular expects **CSV data in a particular structure**: `train/` and `validation/` folders each containing a single `data.csv`, with **no headers**, and the **target column first** in the files.

Here we'll split the raw data snapshot into randomly shuffled training, validation, and test sets - and upload to S3 in the required format:

In [ ]:
df_model_data = full_df.drop(
    columns=[
        # Drop Feature Store metadata fields that aren't relevant to the model:
        "customer_id", "event_time", "write_time", "api_invocation_time", "is_deleted", "row_number"
    ],
    errors="ignore",  # Your DF may not have 'row_number' if you did a simple 'select * from' query
)
df_model_data

# Shuffle and split dataset
train_data, validation_data, test_data = np.split(
    df_model_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(df_model_data)), int(0.9 * len(df_model_data))],
)

# Create CSV files for Train / Validation / Test
train_data.to_csv("data/train.csv", index=False, header=False)
validation_data.to_csv("data/validation.csv", index=False, header=False)
test_data.to_csv("data/test.csv", index=False, header=False)

df_model_data

In [ ]:
model_data_s3uri = f"s3://{bucket_name}/{bucket_prefix}/model-data-ag"

# Upload data to Amazon S3:
train_data_s3uri = model_data_s3uri + "/train/data.csv"
train_data.to_csv(train_data_s3uri, index=False, header=False)
validation_data_s3uri = model_data_s3uri + "/validation/data.csv"
validation_data.to_csv(validation_data_s3uri, index=False, header=False)
test_data_s3uri = model_data_s3uri + "/test/data.csv"
test_data.to_csv(test_data_s3uri, index=False, header=False)

### Train a model

With the data prepared in a compatible format, and the parameters collected, we're ready to run a training job through the SageMaker SDK [Estimator](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) class, which provides a high-level wrapper over the underlying [SageMaker CreateTrainingJob API](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_CreateTrainingJob.html).

The training job runs on **separate, containerized infrastructure** from this notebook:

- **You specify** the number and type of instances, and the IAM permissions with which the job runs (which could be separate from the notebook execution role)
- The job is **independent** from the notebook: The input parameters, logs, metrics, and output artifacts are still available through the APIs even if the notebook disconnects/restarts part way through. (See [Estimator.attach(...)](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Estimator.attach) classmethod for re-attaching to previous/ongoing jobs).
- A range of **other infrastructure parameters** are available like:
    - [SageMaker managed spot](https://docs.aws.amazon.com/sagemaker/latest/dg/model-managed-spot-training.html), to optimize infrastructure costs
    - [Warm pool keep-alive](https://docs.aws.amazon.com/sagemaker/latest/dg/train-warm-pools.html), to speed up start of sequential jobs

In [ ]:
%%time

ag_estimator = sagemaker.estimator.Estimator(
    base_job_name="autogluon",
    role=sgmk_role,  # IAM role for job permissions
    output_path=f"s3://{bucket_name}/{bucket_prefix}/train-output",  # Optional artifact output loc

    image_uri=train_image_uri,  # AutoGluon-Tabular algorithm container
    source_dir=train_source_uri,  # AutoGluon-Tabular script bundle (pre-built)
    model_uri=train_model_uri,  # AutoGluon-Tabular pre-trained artifacts
    entry_point="transfer_learning.py",  # Training script in the source_dir

    hyperparameters=hyperparameters,

    instance_type=training_instance_type,  # Type of compute instance
    instance_count=1,
    max_run=25 * 60,  # Limit job to 25 minutes
)

# Launch a SageMaker Training job by passing the S3 path of the datasets:
ag_estimator.fit({"training": model_data_s3uri})

As well as the logs streamed to the notebook, you can follow the status of the job in:
- The [Training > Training jobs page of the AWS Console for SageMaker](https://console.aws.amazon.com/sagemaker/home?#/jobs)
    - Including links to Amazon CloudWatch console to drill in to job logs and metric graphs
- The Resources > Experiments and trials pane in SageMaker Studio
    - Jobs started without an explicit Experiment configuration will appear under the "Unassigned trial components" folder

### Deploy the model

When the training job is completed successfully, your model is ready to use for inference either in batch or real-time.

For this particular algorithm, the **container URI and script are different** for inference than training, so we need to look up the inference artifacts similarly to training above:

In [ ]:
inference_instance_type = "ml.m5.large"

inference_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    model_id=ag_model_id,
    model_version=ag_model_version,
    image_scope="inference",
    instance_type=inference_instance_type,
)
print(inference_image_uri)

inference_src_uri = script_uris.retrieve(
    model_id=ag_model_id, model_version=ag_model_version, script_scope="inference"
)
print(inference_src_uri)

Although you could deploy in **one line** with `ag_predictor = ag_estimator.deploy(...)`, this encapsulates [multiple steps](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html) of generating a Model, Endpoint Configuration, and Endpoint.

We'll explicitly separate out the model step here, which will be helpful for storing model metadata later:

In [ ]:
ag_model = ag_estimator.create_model(
    image_uri=inference_image_uri,
    source_dir=inference_src_uri,
    entry_point="inference.py",
)

Whether from a Model or direct from an Estimator, setting up a real-time endpoint for the trained model is just one `.deploy(...)` function call as shown below.

> ⏰ This deployment might take **up to 5-10 minutes**, and by default the code will wait for the deployment to complete.

If you like, you can instead:

- Un-comment the `wait=False` parameter (or if you already ran the cell, press the ⏹ "stop" button in the toolbar above)
- Use the [Endpoints page of the SageMaker Console](https://console.aws.amazon.com/sagemaker/home?#/endpoints) to check the status of the deployment
- Skip over the *Evaluation* section below (which won't run until the deployment is complete), and start the Hyperparameter Optimization job - which will take a while to run too, so can be started in parallel

In [ ]:
%%time
ag_predictor = ag_model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    
    # wait=False,  # Remember, predictor.predict() won't work until deployment finishes!

    # We will also turn on data capture here, in case you want to experiment with monitoring later:
    data_capture_config=sagemaker.model_monitor.DataCaptureConfig(
        enable_capture=True,
        sampling_percentage=100,
        destination_s3_uri=f"s3://{bucket_name}/{bucket_prefix}/data-capture",
    ),
)

### Use the endpoint

The [Predictor](https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html) class in the SageMaker SDK provides a high-level Python wrapper for creating and invoking inference endpoints which is useful in notebooks... Although consumer applications can also use the low-level [SageMaker Runtime API](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_Operations_Amazon_SageMaker_Runtime.html) (with [Boto3 in Python](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime.html), or other AWS SDKs in other languages) to avoid this extra library dependency.

SageMaker is a general-purpose ML platform supporting a wide range of use-cases beyond tabular data, so we need to explicitly configure what type of content we're sending in endpoint requests (here CSV) and specifying for the response (here JSON) - via the SDK [serializer](https://sagemaker.readthedocs.io/en/stable/api/inference/serializers.html) and [deserializer classes](https://sagemaker.readthedocs.io/en/stable/api/inference/deserializers.html). You can also define your own classes to fully customize the Python I/O interface of the `predictor.predict()` method and the on-the-wire data format transmitted to the HTTPS endpoint. 

Again, when using a pre-built algorithm, refer to the [algorithm docs](https://docs.aws.amazon.com/sagemaker/latest/dg/autogluon-tabular.html#InputOutput-AutoGluon-Tabular) to see what input and output formats are supported at inference time.

When using `application/json` with the `verbose` flag, AutoGluon-Tabular can return **both** the predicted class labels and the class probabilities, which we'll use below:

In [ ]:
ag_predictor.serializer = sagemaker.serializers.CSVSerializer()
ag_predictor.deserializer = sagemaker.deserializers.JSONDeserializer(
    accept="application/json;verbose"
)

X_test_numpy = test_data.drop(["y"], axis=1).values

model_response = ag_predictor.predict(X_test_numpy)

print("Response keys:", model_response.keys())

# probabilities is (N, 2) with probs for both classes, so convert to 1D probability of cls '1':
probabilities = np.array(model_response["probabilities"], dtype=float)[:, 1].squeeze()
probabilities

> ⚠️ **Note:** The above single `predict()` call makes a single `InvokeEndpoint` request with the **entire test dataset in one batch**. That's fine for a small dataset like this one, but practical use-cases will need to balance between throughput efficiency (large batches reduce communication overhead), endpoint memory requirements, and payload size limits (6MB for real-time endpoints, at the time of writing).

Our model has calculated probability scores (in the interval [0,1]) of a potential customer enrolling for a term deposit, and also assigned labels based on the assumed best threshold:

- 0: The person **will not** enroll
- 1: The person **will** enroll (making them a good candidate for direct marketing)

If we like, we could stitch these predictions back on to the original dataframe to explore performance:

In [ ]:
test_results = pd.concat(
    [
        pd.Series(probabilities, name="y_prob", index=test_data.index),
        pd.Series(model_response["predicted_label"], name="y_pred", index=test_data.index),
        test_data,
    ],
    axis=1,
)
test_results.head()

From this joined data we can calculate standard quality metrics to measure the performance of the classifier.

The utility function below generates a graphical report here in the notebook, but also saves a JSON file in [SageMaker Model Quality Metrics compatible-format](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-metrics.html): Similar to if we'd run a [SageMaker Model Quality Monitoring job](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality.html).

In [ ]:
report = util.reporting.generate_binary_classification_report(
    y_real=test_data["y"].values,
    y_predict_proba=probabilities,
    # Since this model already outputs both labels and probabilities, we can use both:
    y_predict_label=test_results["y_pred"].values,
    # No need for an arbitrary decision threshold:
    # decision_threshold=0.5,
    class_names_list=["Did not enroll", "Enrolled"],
    title="AutoGluon model",
)

# Store the model quality report locally and on Amazon S3:
with open("data/report-autogluon.json", "w") as f:
    json.dump(report, f, indent=2)
model_quality_s3uri = f"s3://{bucket_name}/{bucket_prefix}/{ag_model.name}/model-quality.json"
!aws s3 cp data/report-autogluon.json {model_quality_s3uri}

### Register and share the model

The trained model is already available in the SageMaker APIs to deploy and re-use (you should see it, for example, in the [Models page of the SageMaker Console](https://console.aws.amazon.com/sagemaker/home?#/models)).

However, we can improve discoverability and governance by cataloging it in the [SageMaker Model Registry](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry.html). Here extra metadata can be associated, including I/O formats and the model quality report generated above:

In [ ]:
ag_model.register(
    content_types=["text/csv"],
    response_types=["application/json", "application/json;verbose"],
    model_package_group_name="sm101-dm",
    description="Initial AutoGluon-Tabular model",
    model_metrics=sagemaker.model_metrics.ModelMetrics(
        model_statistics=sagemaker.model_metrics.MetricsSource(
            content_type="application/json",
            s3_uri=model_quality_s3uri,
        ),
    ),
    domain="MACHINE_LEARNING",
    task="CLASSIFICATION",
    sample_payload_url=test_data_s3uri,
)

You can explore and manage your versioned registry model packages in SageMaker Studio from the *SageMaker Resources > Model registry* tab: Including **reviewing and approving** new versions to trigger automated deployments.

## Conclusions

In this notebook, we saw how [**SageMaker Autopilot**](https://docs.aws.amazon.com/sagemaker/latest/dg/autopilot-automate-model-development.html) and the [AutoGluon-Tabular **built-in algorithm**](https://docs.aws.amazon.com/sagemaker/latest/dg/autogluon-tabular.html) can accelerate new ML projects: Helping you build a highly accurate model fast, without lots of manual feature engineering.

- In the case of Autopilot, you don't even need to write code to get started: Just upload your data to Amazon S3 and work through the `Create experiment` UI flow.
- When using built-in algorithms, **refer to the [algorithm's doc pages](https://docs.aws.amazon.com/sagemaker/latest/dg/algos.html)** for important usage info like data formats, and whether multi-instance training parallelism is supported.

We also saw brief intros to how [SageMaker Feature Store](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store.html) can help catalog shared feature data, and how [SageMaker Model Registry](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry.html) helps with tracking and managing trained models. For more information on these MLOps features, you can refer to the documentation and the official [SageMaker notebook examples repository](https://github.com/aws/amazon-sagemaker-examples).

▶️ In [Notebook 2 XGBoost and HPO.ipynb](2%20XGBoost%20and%20HPO.ipynb), you'll see another SageMaker built-in algorithm in action and learn how SageMaker can automatically tune hyperparameters. Follow along to learn more!

## Releasing cloud resources

While training job clusters are shut down automatically when the job stops, inference endpoints stay provisioned until explicitly deleted.

To avoid unnecessary charges, un-comment and run the following code to clean up your AutoGluon model endpoint when finished experimenting. You can also check the [Inference > Endpoints page of the SageMaker console](https://console.aws.amazon.com/sagemaker/home?#/endpoints) for any other running endpoints.

In [ ]:
# ag_predictor.delete_endpoint(delete_endpoint_config=True)